In [1]:
from __future__ import print_function

import argparse
import json
from pprint import pprint
import requests
import sys
import urllib
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [2]:
API_KEY= ""


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'Manhattan'
SEARCH_LIMIT = 20

In [3]:
def request(host, path, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % API_KEY,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(cuisine,offset):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'location': DEFAULT_LOCATION,
        'limit': SEARCH_LIMIT,
        'term': cuisine + "+restaurants",
        'sort_by': 'rating'
    }
    return request(API_HOST, SEARCH_PATH, url_params=url_params)

In [8]:
testObj = search("indian")
pprint(testObj)

Querying https://api.yelp.com/v3/businesses/search ...
{'businesses': [{'alias': 'dadi-ma-home-style-indian-cooking-new-york-2',
                 'categories': [{'alias': 'indpak', 'title': 'Indian'},
                                {'alias': 'seafood', 'title': 'Seafood'}],
                 'coordinates': {'latitude': 40.75557266682884,
                                 'longitude': -73.9687464},
                 'display_phone': '(212) 419-1740',
                 'distance': 406.54828398647186,
                 'id': '3kuaJMPOeTP1O1CK2fyOsA',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/9GQvkKHioN1qNa82I9fG8w/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '246 E 51st',
                              'address2': 'Store 1',
                              'address3': '',
                              'city': 'New York',
                              'country': 'US',
                              'display_address': ['246 E 51

In [19]:
def process(record,cuisine):
    """
    Processess the record and maps them into the required dynamoDB format.
    Args: 
        id : string
        alias : string
        cuisine : string
        phone : string
        image_url : string
        name : string
        rating : number
        review_count : number
        yelp_url : string
        latitude : number
        longitude : number
        address : string
        state : string
        zip_code : string
        
    Returns: 
        dict
    
    
    """
    rec = {}
    rec['id'] = record.get('id',None)
    rec['alias'] = record.get('alias',None)
    rec['cuisine'] = cuisine
    rec['phone'] = record.get('display_phone', None)
    rec['image_url'] = record.get('image_url', None)
    rec['name'] = record.get('name', None)
    rec['rating'] = record.get('rating', None)
    rec['review_count'] = record.get('review_count', None)
    rec['yelp_url'] = record.get('url', None)
    
    coordinates = record.get('coordinates', None)
    if(coordinates != None):
        rec['latitude'] = coordinates.get('latitude', None)
        rec['longitude'] = coordinates.get('longitude', None)
    
    location = record.get('location', None)
    if(location != None):
        address = location.get('display_address', None)
        if(address != None):
            rec['address'] = " ".join(address)
        rec['state'] = location.get('state', None)
        rec['zip_code'] = location.get('zip_code', None)
        
    pprint(rec)
    print()
    
for i in testObj['businesses']:
    # print(type(i)
    process(i,'indian')


# process(test,'indian')

{'address': '246 E 51st Store 1 New York, NY 10022',
 'alias': 'dadi-ma-home-style-indian-cooking-new-york-2',
 'cuisine': 'indian',
 'id': '3kuaJMPOeTP1O1CK2fyOsA',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/9GQvkKHioN1qNa82I9fG8w/o.jpg',
 'latitude': 40.75557266682884,
 'longitude': -73.9687464,
 'name': 'Dadi Ma Home Style Indian Cooking',
 'phone': '(212) 419-1740',
 'rating': 5.0,
 'review_count': 17,
 'state': 'NY',
 'yelp_url': 'https://www.yelp.com/biz/dadi-ma-home-style-indian-cooking-new-york-2?adjust_creative=8VKLDk9ndBeprO5x1ijbTQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=8VKLDk9ndBeprO5x1ijbTQ',
 'zip_code': '10022'}

{'address': '66 Madison Ave New York, NY 10016',
 'alias': 'cloves-indian-cuisine-new-york-2',
 'cuisine': 'indian',
 'id': 'IysZ64t48AedrhWfz3WJGQ',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/6A5nzyLwmGAcXF1KPn94uQ/o.jpg',
 'latitude': 40.74367,
 'longitude': -73.98612,
 'name': 'Cloves Indian Cuisine',
 'p

In [ ]:
import boto3

# Create a DynamoDB client
dynamodb = boto3.resource('dynamodb')

# Create a table object
table = dynamodb.Table('my-table')

# Create an item to insert
item = {
    'name': 'John Doe',
    'age': 30,
    'city': 'Seattle'
}

# Insert the item into the table
table.put_item(Item=item)

# Print a confirmation message
print('Item inserted successfully!')

'2024-02-08T17:18:45.795748'